### CNN model

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import matplotlib.pyplot as plt

In [15]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [16]:
train_data = datasets.USPS(root='data', train=True, download=True, transform=transform)
test_data = datasets.USPS(root='data', train=False, download=True, transform=transform)

In [17]:
train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers = 2)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False, num_workers = 2)

In [18]:
data , images = next(iter(train_loader))

In [19]:
first_image = data[0:1]

In [20]:
class CNN_with_MLP(nn.Module):
  def __init__(self):
    super(CNN_with_MLP,self).__init__()

    self.conv1 = nn.Sequential(
        nn.Conv2d(in_channels = 1, out_channels=32,kernel_size=3),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )

    self.conv2 = nn.Sequential(
        nn.Conv2d(in_channels = 32, out_channels=64,kernel_size=3),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )

    self.FC1 = nn.Linear(256,128)
    self.FC2 = nn.Linear(128,10)

  def forward(self,X):
    X = self.conv1(X)
    X = self.conv2(X)

    flatten = nn.Flatten()
    relu = nn.ReLU()

    X = flatten(X)
    X = self.FC1(X)
    X = relu(X)

    X = self.FC2(X)

    return X



In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [22]:
cnn_model = CNN_with_MLP()
# cnn_model.to(device)

In [23]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_model.parameters(), lr=0.001)

In [24]:
cnn_writer = SummaryWriter('logs/cnn')

In [25]:
def evaluate_model(model, dataloader):
    model.eval()
    all_labels = []
    all_preds = []
    with torch.no_grad():
        for inputs, labels in dataloader:
            # inputs = inputs.to(device)
            # labels = labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            all_labels.extend(labels.numpy())
            all_preds.extend(predicted.numpy())
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    cm = confusion_matrix(all_labels, all_preds)
    return accuracy, precision, recall, cm

In [26]:
for epoch in range(20):
        cnn_model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            # inputs = inputs.to(device)
            # labels = labels.to(device)
            optimizer.zero_grad()
            outputs = cnn_model(inputs)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()


        cnn_writer.add_scalar('Loss/train', running_loss / len(train_loader), epoch)


        cnn_accuracy, cnn_precision,cnn_recall, cnn_cm = evaluate_model(cnn_model, test_loader)


        cnn_writer.add_scalar('Accuracy/test', cnn_accuracy, epoch)
        cnn_writer.add_scalar('Precision/test',cnn_precision, epoch)
        cnn_writer.add_scalar('Recall/test', cnn_recall, epoch)

        print(f"Epoch {epoch} Loss: {running_loss / len(train_loader)}, Accuracy: {cnn_accuracy}, Precision: {cnn_precision}, Recall: {cnn_recall}")


Epoch 0 Loss: 0.6518504053615687, Accuracy: 0.8988540109616343, Precision: 0.8932895155559756, Recall: 0.8929336171936308
Epoch 1 Loss: 0.1760012608291091, Accuracy: 0.9292476332835077, Precision: 0.9295094279193016, Recall: 0.9198088764801898
Epoch 2 Loss: 0.11237915969461922, Accuracy: 0.9302441454907823, Precision: 0.928160807412076, Recall: 0.9218634133381298
Epoch 3 Loss: 0.08371601714099054, Accuracy: 0.9402092675635276, Precision: 0.9375138051406797, Recall: 0.9337914534275018
Epoch 4 Loss: 0.06643281937188779, Accuracy: 0.9491778774289985, Precision: 0.9454421100035864, Recall: 0.9439149893815708
Epoch 5 Loss: 0.050916062105291836, Accuracy: 0.9466865969108121, Precision: 0.9456752522396952, Recall: 0.9404349938183924
Epoch 6 Loss: 0.04447768539498691, Accuracy: 0.9456900847035377, Precision: 0.9418867628221049, Recall: 0.9404950190859171
Epoch 7 Loss: 0.03454619239078296, Accuracy: 0.9481813652217239, Precision: 0.9445729299897796, Recall: 0.9431378396688939
Epoch 8 Loss: 0.02

In [27]:
cnn_writer.close()

In [28]:
print("CNN Results:")
print("Accuracy:", cnn_accuracy)
print("Precision:", cnn_precision)
print("Recall:", cnn_recall)
print("Confusion Matrix:")
print(cnn_cm)


CNN Results:
Accuracy: 0.9431988041853513
Precision: 0.942422202722463
Recall: 0.9383069167066532
Confusion Matrix:
[[349   0   2   0   3   3   0   0   2   0]
 [  0 259   0   0   4   0   1   0   0   0]
 [  1   0 178   0   6   4   0   1   8   0]
 [  0   0   0 140   1  21   0   0   4   0]
 [  0   2   2   1 190   3   0   1   1   0]
 [  1   0   0   2   0 157   0   0   0   0]
 [  0   0   0   0   5  10 153   0   2   0]
 [  0   0   1   1   6   1   0 136   1   1]
 [  1   0   0   0   1   1   0   0 163   0]
 [  0   1   0   1   2   0   0   0   5 168]]


In [29]:
%load_ext tensorboard

In [30]:
!kill 1850

/bin/bash: line 1: kill: (1850) - No such process


In [ ]:
tensorboard --logdir=logs


### MLP model

In [32]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = torch.flatten(x, 1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [39]:
mlp_model = MLP(input_size=256, hidden_size=128, num_classes=10)


In [40]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_model.parameters(), lr=0.001)

In [41]:
mlp_writer = SummaryWriter('logs/mlp')

In [42]:
def evaluate_model(model, dataloader):
    model.eval()
    all_labels = []
    all_preds = []
    with torch.no_grad():
        for inputs, labels in dataloader:
            # inputs = inputs.to(device)
            # labels = labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            all_labels.extend(labels.numpy())
            all_preds.extend(predicted.numpy())
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    cm = confusion_matrix(all_labels, all_preds)
    return accuracy, precision, recall, cm

In [43]:
for epoch in range(20):
        mlp_model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            # inputs = inputs.to(device)
            # labels = labels.to(device)
            optimizer.zero_grad()
            outputs = mlp_model(inputs)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()


        mlp_writer.add_scalar('Loss/train', running_loss / len(train_loader), epoch)


        mlp_accuracy, mlp_precision,nlp_recall, mlp_cm = evaluate_model(mlp_model, test_loader)


        cnn_writer.add_scalar('Accuracy/test', cnn_accuracy, epoch)
        cnn_writer.add_scalar('Precision/test',cnn_precision, epoch)
        cnn_writer.add_scalar('Recall/test', cnn_recall, epoch)

        print(f"Epoch {epoch} Loss: {running_loss / len(train_loader)}, Accuracy: {cnn_accuracy}, Precision: {cnn_precision}, Recall: {cnn_recall}")


Epoch 0 Loss: 2.3307049148961116, Accuracy: 0.9431988041853513, Precision: 0.942422202722463, Recall: 0.9383069167066532
Epoch 1 Loss: 2.330693663212291, Accuracy: 0.9431988041853513, Precision: 0.942422202722463, Recall: 0.9383069167066532
Epoch 2 Loss: 2.330696833761115, Accuracy: 0.9431988041853513, Precision: 0.942422202722463, Recall: 0.9383069167066532
Epoch 3 Loss: 2.330665427341796, Accuracy: 0.9431988041853513, Precision: 0.942422202722463, Recall: 0.9383069167066532
Epoch 4 Loss: 2.330706602648685, Accuracy: 0.9431988041853513, Precision: 0.942422202722463, Recall: 0.9383069167066532
Epoch 5 Loss: 2.33070254953284, Accuracy: 0.9431988041853513, Precision: 0.942422202722463, Recall: 0.9383069167066532
Epoch 6 Loss: 2.330670689281664, Accuracy: 0.9431988041853513, Precision: 0.942422202722463, Recall: 0.9383069167066532
Epoch 7 Loss: 2.3306830640424763, Accuracy: 0.9431988041853513, Precision: 0.942422202722463, Recall: 0.9383069167066532
Epoch 8 Loss: 2.330690720625091, Accura

In [44]:
mlp_writer.close()

In [45]:
print("CNN Results:")
print("Accuracy:", cnn_accuracy)
print("Precision:", cnn_precision)
print("Recall:", cnn_recall)
print("Confusion Matrix:")
print(cnn_cm)


CNN Results:
Accuracy: 0.9431988041853513
Precision: 0.942422202722463
Recall: 0.9383069167066532
Confusion Matrix:
[[349   0   2   0   3   3   0   0   2   0]
 [  0 259   0   0   4   0   1   0   0   0]
 [  1   0 178   0   6   4   0   1   8   0]
 [  0   0   0 140   1  21   0   0   4   0]
 [  0   2   2   1 190   3   0   1   1   0]
 [  1   0   0   2   0 157   0   0   0   0]
 [  0   0   0   0   5  10 153   0   2   0]
 [  0   0   1   1   6   1   0 136   1   1]
 [  1   0   0   0   1   1   0   0 163   0]
 [  0   1   0   1   2   0   0   0   5 168]]


In [46]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [48]:
!kill 1705

In [ ]:
tensorboard --logdir=logs
